# <a id="0">Wine Data Exercises (Part 2) with pipeline and column transformer</a>

In this notebook, we will review basic steps of exploratory data analysis following the example in the EDA-PIPELINE,ipynb example. We will work with the wine data set __winequality-white.csv__ provided in the data folder. 

__Dataset schema:__ 
   - fixed acidity
   - volatile acidity
   - citric acid
   - residual sugar
   - chlorides
   - free sulfur dioxide
   - total sulfur dioxide
   - density
   - pH
   - sulphates
   - alcohol

   Output variable (based on sensory data): 
   - quality (score between 0 and 10)

In [1]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")
  
df = pd.read_csv('../data/winequality-white.csv', sep=';')

print('The shape of the dataset is:', df.shape)

The shape of the dataset is: (4898, 12)


We will look at number of rows, columns and some simple statistics of the dataset using [df.info()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html)

In [2]:
df.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4897 non-null   float64
 3   residual sugar        4896 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4897 non-null   float64
 6   total sulfur dioxide  4896 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4897 non-null   float64
 9   sulphates             4897 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


Create a catagorical feature column

In [3]:
bin_labels_5 = ['poor', 'average', 'ok', 'good', 'best']
df['fixed acidity group'] = pd.qcut(df['fixed acidity'],
                              q=[0, .2, .4, .6, .8, 1],
                              labels=bin_labels_5)

In [4]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,fixed acidity group
0,7.0,0.27,0.36,NaN,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,ok
1,6.3,0.30,0.34,NaN,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,average
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,best
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,good
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,good


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   fixed acidity         4898 non-null   float64 
 1   volatile acidity      4898 non-null   float64 
 2   citric acid           4897 non-null   float64 
 3   residual sugar        4896 non-null   float64 
 4   chlorides             4898 non-null   float64 
 5   free sulfur dioxide   4897 non-null   float64 
 6   total sulfur dioxide  4896 non-null   float64 
 7   density               4898 non-null   float64 
 8   pH                    4897 non-null   float64 
 9   sulphates             4897 non-null   float64 
 10  alcohol               4898 non-null   float64 
 11  quality               4898 non-null   int64   
 12  fixed acidity group   4898 non-null   category
dtypes: category(1), float64(11), int64(1)
memory usage: 464.3 KB


In [6]:
df.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             1
residual sugar          2
chlorides               0
free sulfur dioxide     1
total sulfur dioxide    2
density                 0
pH                      1
sulphates               1
alcohol                 0
quality                 0
fixed acidity group     0
dtype: int64

In [7]:
print(df.columns)

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality', 'fixed acidity group'],
      dtype='object')


In [8]:
model_features = df.columns.drop('quality')
model_target = 'quality'

print('Model features: ', model_features)
print('Model target: ', model_target)

Model features:  Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'fixed acidity group'],
      dtype='object')
Model target:  quality


In [9]:
import numpy as np
numerical_features_all = df[model_features].select_dtypes(include=np.number).columns
print('Numerical columns:',numerical_features_all)

print('')

categorical_features_all = df[model_features].select_dtypes(include='category').columns
print('Categorical columns:',categorical_features_all)


Numerical columns: Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol'],
      dtype='object')

Categorical columns: Index(['fixed acidity group'], dtype='object')


In [10]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.1, shuffle=True, random_state=42)

## Data Processing with Pipeline
 
__Part 1.__ Build a pipeline that has two pre-processors

- One is to impute the missing values with the mean using sklearn's SimpleImputer, scale the numerical features to have similar orders of magnitude by bringing them into the 0-1 range with sklearn's MinMaxScaler, for numerical features 

- One is to use one hot encoding to encode the catagorical feature. Note here even the feature `fixed acidity group` is ordinal, we apply one hot encoding for the sake of practice.   

Then we add a decision tree estimator to form the pipeline. Visualize pipeline. 

__Part 2.__ Test the pipeline on the training data, then on the test data.

__Part 3.__ Use Grid Search to tune the hyper-parameter for the decision tree estimator. You may use a grid like this

        param_grid={'dt__max_depth': [100, 200, 300],#, 50, 75, 100, 125, 150, 200, 250], 
            'dt__min_samples_leaf': [5, 10, 15],#, 25, 30],
            'dt__min_samples_split': [2, 5, 15]#, 25, 30, 45, 50]
        }
